# Cái này là để cài đặt folder thành pakege để gọi imoport mà ko lỗi á. Nó tự sinh ra file 'signature_verification.egg-info' chứa thông tin tác giả của mìnhmình

In [1]:
%cd ..

# Cài đặt package
!pip install -e .

# Import module
from utils.helpers import load_config
from losses.triplet_loss import tripletLoss
from models.triplet_network import TripletNetwork

print("Package installed and imported successfully!")

d:\Project_Management\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication


C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///D:/Project_Management/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of signature-verification to determine which version is compatible with other requirements. This could take a while.


ERROR: Could not find a version that satisfies the requirement pyymal (from signature-verification) (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pyymal


Package installed and imported successfully!


In [ ]:
import os
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Project root: d:\


In [ ]:
#Load data, bằng các lớp trong folder dataset, tui nghĩa nên làm 2 cái loader riêng cho 2 model do cấu trúc khác nhau
#Triplet cần 3 ảnh anchor, positive, negative, vậy nên trong Loader sau khi load mỗi batch chứa 1 tuple 3 ảnh
dataLloader = (1,2,3) #Tui viết cho có

In [3]:
#Nayf ae xem sửa thửthử
config = load_config(r'D:\Project_Management\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\configs\config_triplet.yaml')
print(config)

{'model': {'backbone': 'resnet34', 'feature_dim': 512}, 'training': {'batch_size': 32, 'learning_rate': 0.001, 'optimizer': 'adam', 'num_epochs': 50, 'margin': 0.5, 'scheduler': 'cosine'}, 'dataset': {'train_data_path': './data/triplet/train/', 'val_data_path': './data/triplet/val/', 'input_size': '(220,150)'}, 'device': 'cuda', 'logging': {'log_dir': './logs/', 'checkpoint_dir': './checkpoints/', 'save_freq': 5}}


In [ ]:
#Tiếp theo định nghĩa model từ con config
model = TripletNetwork(config['model']['backbone'], config['model']['feature_dim'])

optimizer = optim.Adam(model.parameters(), lr=config['training']['learning_rate'])
device = torch.device(config['device'] if torch.cuda.is_available() else "cpu")
model.to(device)

AE xem thử rồi cấu trúc ổn không.

In [ ]:
#Train kiểu kiểu như vầy ae cứ xem tham thảo

num_epochs = config['train']['num_epochs']
checkpoint_dir = config['logging']['checkpoint_dir']
save_freq = config['logging']['save_freq']

# Tạo thư mục lưu checkpoint nếu chưa có
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for anchor, positive, negative in dataloader:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        anchor_feat, positive_feat, negative_feat = model(anchor, positive, negative)

        loss = tripletLoss(anchor_feat, positive_feat, negative_feat, margin=config['loss']['margin']) 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch}/{num_epochs}] - Loss: {avg_loss:.4f}")

    # === SAVE CHECKPOINT ===
    if epoch % save_freq == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")
